In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load the data
df = pd.read_excel("./data/FINAL_DATASET.xlsx")

for name in df.columns:
    newname = name
    # remove alla characted within two []
    while "[" in newname:
        start = newname.find("[")
        end = newname.find("]")
        newname = newname[:start] + newname[end+1:]
    # remove end spaces and beginning spaces
    newname = newname.strip()
    # remove alla mathematical oipreators caharacters and spaces
    newname = newname.replace(" ", "_")
    newname = newname.replace(">", "gt")
    newname = newname.replace("<", "lt")
    newname = newname.replace("=", "eq")
    newname = newname.replace("+", "plus")
    newname = newname.replace("-", "minus")
    newname = newname.replace("*", "times")
    newname = newname.replace("/", "div")
    # rename columns in the dataframe
    df.rename(columns={name: newname}, inplace=True)

In [2]:
labelname  = "d33"
materialfeatures = ['octahedra_volume_min', 'octahedra_volume_max', 'octahedra_volume_avg',
       'octahedra_meanangle_axis_min', 'octahedra_meanangle_axis_max',
       'octahedra_meanangle_axis_avg', 'tilt_BOB_ip_min', 'tilt_BOB_ip_max',
       'tilt_BOB_ip_avg', 'tilt_BOB_oop_min', 'tilt_BOB_oop_max',
       'tilt_BOB_oop_avg', 'spageGroup_no', 'lattice_a', 'lattice_b',
       'lattice_c', 'lattice_alfa', 'lattice_beta', 'lattice_gamma',
       'volume_uc', 'volume_uc_per_atom', 'volume_ratio_ucVSoctahedra',
       'tolerance_factor', 'ratio_outVSinplaneAVG', 'ratio_outVSinplanemin',
       'ratio_outVSinplanemax', 'bond_lengthAA_min', 'bond_lengthAA_max',
       'bond_lengthAA_avg', 'bond_lengthAB_min', 'bond_lengthAB_max',
       'bond_lengthAB_avg', 'bond_lengthAO_min', 'bond_lengthAO_max',
       'bond_lengthAO_avg', 'bond_lengthBO_min', 'bond_lengthBO_max',
       'bond_lengthBO_avg', 'bond_lengthBB_min', 'bond_lengthBB_max',
       'bond_lengthBB_avg', 'is_magnetic', 'is_metallic', 'is_perovskite', 
       'P_0_z', 'P_0_z_muCdivcm2',]
atomicfeatures = ['A_Z', 'A_group', 'A_valence',
       'A_vecdivZ', 'A_n_d', 'A_atomic_volume_pymatgen', 'A_Rdce', 'A_Rdve',
       'A_rs_max', 'A_rd_max', 'A_IE_ionization_energy',
       'A_EA_electron_affinity', 'A_Mulliken', 'A_Pauling',
       'A_MartynovminusBatsanov', 'A_atomic_radius_rahm', 'A_vdw_radius_uff',
       'A_ionic_radius', 'B_Z', 'B_group', 'B_valence', 'B_vecdivZ', 'B_n_d',
       'B_atomic_volume_pymatgen', 'B_Rdce', 'B_Rdve', 'B_rs_max', 'B_rd_max',
       'B_IE_ionization_energy', 'B_EA_electron_affinity', 'B_Mulliken',
       'B_Pauling', 'B_MartynovminusBatsanov', 'B_atomic_radius_rahm',
       'B_vdw_radius_uff', 'B_residual_d', 'B_ionic_radius']

Y = df[labelname]
X = df[materialfeatures + atomicfeatures]
X_mat = df[materialfeatures]
X_atm = df[atomicfeatures]

In [ ]:
CORRCUT = 0.95
# compute and plot correlation matrix
corr = X.corr()
# remove highly correlated values
# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] >CORRCUT)]
print("Dropping features: ", to_drop)
# Drop features
X = X.drop(X[to_drop], axis=1)

corr_mat = X_mat.corr()
# remove highly correlated values
# Select upper triangle of correlation matrix
upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))
print("Dropping features: ", to_drop)
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > CORRCUT)]
# Drop features
X_mat = X_mat.drop(X_mat[to_drop], axis=1)

corr_atm = X_atm.corr()
# remove highly correlated values
# Select upper triangle of correlation matrix
upper = corr_atm.where(np.triu(np.ones(corr_atm.shape), k=1).astype(np.bool))
print("Dropping features: ", to_drop)
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > CORRCUT)]
# Drop features
X_atm = X_atm.drop(X_atm[to_drop], axis=1)



In [4]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)
X_mat = StandardScaler().fit_transform(X_mat)
X_atm = StandardScaler().fit_transform(X_atm)

In [ ]:
from sklearn.decomposition import PCA

sumofvar = []
ratioofvar = []

first80 = False
firtt90 = False
for nc in range(1, X.shape[1]):
    pca = PCA(n_components=nc)
    pcs = pca.fit_transform(X)
    sumofvar.append(sum(pca.explained_variance_ratio_))
    #print("Explained variance ratio for ", nc, " components: ", pca.explained_variance_ratio_)
    ratioofvar.append(pca.explained_variance_ratio_[-1])
    if sumofvar[-1] > 0.8 and not first80:
        print("Number of components for 80% explained variance: ", nc, " of ", X.shape[1])
        first80 = True
    if sumofvar[-1] > 0.9 and not firtt90:
        print("Number of components for 90% explained variance: ", nc, " of ", X.shape[1])
        firtt90 = True
    #print("Explained variance ratio for ", nc, " components: ", pca.explained_variance_ratio_)
    #print("Explained variance for ", nc, " components: ", pca.explained_variance_)
    #print("Sum of explained variance for ", nc, " components: ", sum(pca.explained_variance_ratio_))

# plot the explained variance ratio
plt.plot(sumofvar)
plt.xlabel("Number of components")
plt.ylabel("Explained variance")
plt.title("Explained variance ratio")
plt.show()

# plot the explained variance ratio
plt.plot(ratioofvar)
plt.xlabel("Number of components")
plt.ylabel("Explained variance")
plt.title("Explained variance ratio")
plt.show()
